Making the models

In [1]:
import pandas as pd
import numpy as np
import vaex
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
# from sklearn.impute import SimpleImputer
# from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder
import joblib

In [2]:
data =pd.read_csv('Doceree-HCP_Train.csv', encoding='latin-1')
test =pd.read_csv('Doceree-HCP-Test.csv', encoding='latin-1')

In [3]:
# Assuming 'data' is pandas DataFrame containing string columns
label_encoder = OrdinalEncoder()

for column in data.columns:
    if data[column].dtype == 'object':
        # Encode the string data to numerical values
        data[column] = label_encoder.fit_transform(data[column])

# The Same thing for test_dataset
for column in test.columns:
    if test[column].dtype == 'object':
        test[column] = label_encoder.fit_transform(test[column])

ValueError: Expected 2D array, got 1D array instead:
array=['Desktop' 'Desktop' 'Desktop' ... 'Desktop' 'Mobile' 'Mobile'].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [ ]:
vaex_data=vaex.from_pandas(data)
vaex_test=vaex.from_pandas(test)

In [ ]:
vaex_data.export_hdf5('Doceree-HCP_Train.hdf5')
vaex_test.export_hdf5('Doceree-HCP-Test.hdf5')

In [ ]:
# # imputing
# imputer = SimpleImputer(strategy='mean')
# # txo_imputer = SimpleImputer(strategy='mean')

In [ ]:
vaex_df=vaex.open('Doceree-HCP_Train.hdf5')
# Iterate over each column in the DataFrame
for column in vaex_df.columns:
    # Fill missing values with 0 for the current column
    vaex_df[column].fillna(0, inplace=True)

In [ ]:
vaex_df = vaex_df.shuffle(random_state=42)  # Shuffle the DataFrame with a fixed random state
df_train,df_test = vaex_df.ml.train_test_split(test_size=0.2)  # Split into train and test sets

# df_train.shape

In [ ]:
features=['DEVICETYPE', 'PLATFORM_ID', 'BIDREQUESTIP', 'USERPLATFORMUID', 'USERCITY', 'USERZIPCODE', 'USERAGENT', 'PLATFORMTYPE', 'CHANNELTYPE', 'URL', 'KEYWORDS']
target='IS_HCP'
txo_features = ['DEVICETYPE', 'PLATFORM_ID', 'BIDREQUESTIP', 'USERPLATFORMUID', 'USERCITY', 'USERZIPCODE', 'USERAGENT', 'PLATFORMTYPE', 'CHANNELTYPE', 'URL', 'KEYWORDS']
txo_target='TAXONOMY'

In [ ]:
# Create the RandomForestRegressor model
model = RandomForestClassifier(n_estimators=100)

In [ ]:
# Fill missing values with a specific value (e.g., 0)
df_train = df_train.fillna(0)
df_test = df_test.fillna(0)

In [ ]:
# Extract the features and target arrays from the DataFrames
X_train = df_train[features].values
# X_train.shapes
# print(X_train)
y_train = df_train[target].values
y_train = y_train.ravel()
txo_train = y_train #to use in futuer

X_test = df_test[features].values
y_test = df_test[target].values
y_test = y_test.ravel()
txo_test = y_test #to use in futuer

print(X_train.shape,y_train.shape)

In [ ]:
# # impute NaN values
# X_train[np.isnan(X_train)] = 0

# y_train = pd.Series(y_train)
# # Fill missing values in y_train with 0
# y_train = y_train.fillna(0)

# print(X_train.shape , y_train.shape)

In [ ]:
# Fit the model
model.fit(X_train, y_train)

In [ ]:
# making predictions
# X_test[np.isnan(X_test)] = 0
predictions = model.predict(X_test)
# print(predictions)

accuracy = accuracy_score(y_test,predictions)
# Print the values
print("IS_HCP Model Evaluation:")
print("Accuracy is :", accuracy)
print(X_train.shape,y_train.shape)

Txo Model

In [ ]:
# # creating mask
# mask1 = y_train.astype(np.int64)
# mask2 = y_test.astype(np.int64)

In [ ]:
# Extract the features and target arrays from the DataFrames
X_train = df_train[txo_features].values
y_train = df_train[txo_target].values
# print(X_train.shape)

X_test = df_test[txo_features].values
y_test = df_test[txo_target].values

# # mask = predictions == 1
# X_train = X_train[mask1]
# print(X_train.shape)

# y_train = y_train[mask1]
# print(y_train)

# # mask = predictions == 1
# X_test = X_test[mask2]
# y_test = y_test[mask2]


# y_test = y_test.ravel()

In [ ]:
# Creating model
txo_model = RandomForestClassifier(n_estimators=100)

In [ ]:
# impute NaN values
# X_train[np.isnan(X_train)] = 0

# y_train = pd.Series(y_train)
# # Fill missing values in y_train with 0
# y_train = y_train.fillna(0)

# print(X_train.shape,y_train.shape)
# X_train

In [ ]:
# Fit model
txo_model.fit(X_train,y_train)

In [ ]:
# Model Evaluation
# X_test[np.isnan(X_test)] = 0

t_prediction = txo_model.predict(X_test)

In [ ]:
print(t_prediction)

In [ ]:
# Calculate and print the evaluation metrics 0.5745445036738107 24.9261978021978
accuracy = accuracy_score(y_test,t_prediction)

print("\nTAXONOMY Model Evaluation:")
print("Accurcy is :", accuracy)

Saving the models

In [ ]:
# Save the trained model
joblib.dump(model, 'trained_model.pkl')
joblib.dump(txo_model, 'txo_trained_model.pkl')

Using the models for prediction

Part-1

In [ ]:
## Load the trained model
prediction_model = joblib.load('trained_model.pkl')

# Load the vaex DataFrame for prediction
df_prediction = vaex.open('Doceree-HCP-Test.hdf5')
# Iterate over each column in the DataFrame
for column in df_prediction.columns:
    # Fill missing values with 0 for the current column
    df_prediction[column].fillna(0, inplace=True)

df_prediction = df_prediction.fillna(0)

X_prediction = df_prediction[features].values
user_ids = df_prediction['ID'].values

# Make predictions
predictions = prediction_model.predict(X_prediction)

# Apply threshold for classification if needed
# threshold = 0.461  //use previous used value of threshold

# class_predictions = np.where(predictions >= threshold, 1, 0)
IS_HCP = predictions

Part-2

In [ ]:
# Loading Txo model
txo_prediction_model = joblib.load('txo_trained_model.pkl')

# Making dataframe for txo model
X_prediction = X_prediction[IS_HCP == 1]
ids = df_prediction['ID'].values
ids = ids[IS_HCP == 1]

In [ ]:
# X_prediction = df_prediction[txo_features].values

In [ ]:
#make prediction
prediction = txo_prediction_model.predict(X_prediction)

In [ ]:
# if prediction.dtype == int:
prediction = label_encoder.inverse_transform(prediction)

In [ ]:
zip(X_prediction,prediction)

# creating mask
mask = np.isin(user_ids,ids)

In [ ]:
prediction = np.resize(prediction,len(mask))


In [ ]:
print(prediction)

In [ ]:
txo_prediction = np.where(mask, prediction,"")

# print(user_ids.shape,txo_prediction.shape,IS_HCP.shape)

In [ ]:
# Create a DataFrame with the user IDs (userplatformuid) and the predicted HCP values and taxonomy predictions

predictions = pd.DataFrame({'ID': user_ids,'taxonomy': txo_prediction, 'IS_HCP': IS_HCP})

# Save the DataFrame to an Excel file
predictions.to_excel('predictions.xlsx', index=False)